In [14]:
!pip install beautifulsoup4

In [15]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import re

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
def truncate_at_read_more(text):
    read_more_index = text.find("(Read more")
    return text[:read_more_index] if read_more_index != -1 else text

In [18]:
base_url = "https://www.newser.com"

urls, headlines, summaries, articles = [], [], [], []

for page_num in range(1, 8):
    url = f"https://www.newser.com/section/5/money-news-headlines.html?PageNum={page_num}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all('a', class_='aGridSquareLink')
    for link in links:
        href = link.get('href')
        full_url = urljoin(base_url, href)

        full_response = requests.get(full_url)
        full_soup = BeautifulSoup(full_response.text, 'html.parser')

        headline_element = full_soup.find('div', {'id': 'divTitle'})
        summary_element = full_soup.find('div', {'id': 'divDeck'})
        article_body_element = full_soup.find('div', {'id': 'divStoryBody'})

        if headline_element:
            headline = headline_element.text.strip()
            summary = summary_element.text.strip() if summary_element else "Summary not found"
            article_content = article_body_element.text.strip() if article_body_element else ""

            article_content = truncate_at_read_more(article_content)
            article_content = ' '.join(article_content.split())

            urls.append(full_url)
            headlines.append(headline)
            summaries.append(summary)
            articles.append(article_content)
            print(full_url)

data = {'URL': urls, 'Headline': headlines, 'Summary': summaries, 'Article': articles}
df = pd.DataFrame(data)

df.to_csv('/content/drive/My Drive/article_dataset.csv', index=False, encoding='utf-8')

https://www.newser.com/story/345606/prosecutors-vet-faked-disability-for-660k-in-benefits.html
https://www.newser.com/story/345612/tech-stocks-slow-market.html
https://www.newser.com/story/345593/its-been-a-brutal-stretch-for-the-media-industry.html
https://www.newser.com/story/345592/its-tough-out-there-for-us-renters.html
https://www.newser.com/story/345575/americans-are-just-not-wired-to-save-money.html
https://www.newser.com/story/345562/markets-rise-despite-rough-day-for-tesla.html
https://www.newser.com/story/345543/new-gdp-report-is-surprisingly-strong.html
https://www.newser.com/story/308846/state-ags-announce-landmark-26b-opioid-settlement.html
https://www.newser.com/story/345511/netflix-leaps-107-after-surge-in-subscribers.html
https://www.newser.com/story/345496/in-n-outs-unprecedented-move-its-closing-a-restaurant.html
https://www.newser.com/story/345491/the-tech-bloodletting-continues-with-cuts-at-ebay.html
https://www.newser.com/story/345485/new-york-city-to-cancel-2b-in-